In [2]:
import pandas as pd
import numpy as np

In [3]:
## taking the y variable with columns: country and unadjusted pay gap
y =  pd.read_csv("lonbg.csv")

In [5]:
y

,country,unadjusted_gender_pay_gap
0,Latvia,19.0
1,Austria,18.3
2,Czechia,18.0
3,Hungary,17.8
4,Germany,17.6
5,Estonia,16.9
6,Finland,16.8
7,Slovakia,15.7
8,Denmark,14.0
9,Greece,13.6


## Education

In [4]:
## reading the data for the x variable education
education = pd.read_csv("tertiary_education.csv")

In [5]:
## filtering the data for the year 2023
education["Tertiary_Education(%)"] = education["2023"]
education = education.drop(["2023"], axis=1)

In [ ]:
## Checking the countries which are not in the y variable
education[~education["country"].isin(y["country"])].dropna()

,country,Tertiary_Education(%)
0,EU,84.1
31,Serbia,93.7
32,Bosnia and Herzegovina,91.9
33,Turkey,76.5


In [6]:
## adding the education data and the y variable in a new dataframe
df = pd.merge(y, education, how="left", left_on="country", right_on="country")

## GDP per capita PPS

In [7]:
## reading the data for the x variable GDP
gdp = pd.read_csv("GDP per capita in PPS_tec00114.csv")

In [8]:
## selecting the columns of interest and filtering for the year 2023
col=["Geopolitical entity (reporting)", "TIME_PERIOD", "OBS_VALUE"]
gdp = gdp[col][gdp["TIME_PERIOD"] == 2023]

## renaming the columns
gdp["GDP_perCapitaPPS"] = gdp["OBS_VALUE"]
gdp = gdp.drop(["OBS_VALUE", "TIME_PERIOD"], axis=1)

In [9]:
## adding gdp data to the dataframe with education and y variable
df = pd.merge(df, gdp, how="left", left_on="country", right_on="Geopolitical entity (reporting)").drop(columns="Geopolitical entity (reporting)", axis=1)

## Region

In [ ]:
text = "Northern Europe: Denmark, Isle of Man, Estonia, Faroe Islands (Denmark), Finland (incl. Åland Islands), United Kingdom, Guernsey, Ireland, Jersey, Latvia, Lithuania, Norway, Svalbard and Jan Mayen, Iceland, Sweden./n Eastern Europe: Bulgaria, Hungary, Moldova, Ukraine, Poland, Romania, Russia (West of the Urals), Slovakia, Czech Republic, Belarus./n Southern Europe: Albania, Andorra, Bosnia-Herzegovina, Gibraltar, Greece, Italy, Kosovo, Croatia, Macedonia, Malta, Montenegro, Portugal, San Marino, Serbia, Slovenia, Spain, Turkey (European part), Vatican City./n Western Europe: Belgium, Germany, France, Liechtenstein, Luxembourg, Monaco, Netherlands, Austria, Switzerland."

In [10]:
## reading the data for the x variable geographical regions
regions = pd.read_csv('world_regions.csv')

In [11]:
## choosing the columns of interest and renaming them
col = ['Region Name', 'Sub-region Name', 'Country or Area']
regions = regions[col].drop(["Region Name"], axis=1).rename(columns={"Sub-region Name": "Geographical_region", "Country or Area": "country"})


# df = pd.merge(df, regions, how="left", left_on="country", right_on="country").drop(columns="country", axis=1)
# regions[col][regions["Country or Area"].isin(df["country"])]
# regions[col][regions['Region Name'] == 'Europe']

In [12]:
## adding the geographical regions data to the dataframe
df = pd.merge(df, regions, how="left", left_on="country", right_on="country")

## Maternity leave

In [13]:
## reading the data for the x variable maternity leave entitlements
maternity = pd.read_csv('PF2_5_Trends_in_leave_entitlements_around_childbirth__editting.csv')

In [14]:
## choosing the columns of interest i.e., country and maternity leave weeks which tells about number of weeks of maternity leave a mother is entitled to before and after childbirth in the year 2023
col = ['Country', 'Maternity_weeks']
maternity = maternity[maternity["Year"] == 2023][col]

# maternity[(maternity['Year'] == 2023) | (maternity['Maternity_weeks'] == 'Number of weeks of maternity leave')][maternity["Country"].isin(df["country"]) | (maternity['Maternity_weeks'] == 'Number of weeks of maternity leave')]

In [41]:
## these are the countries which are not in the maternity weeks variable but is in the y variable
## these are the countries maternity leave data is to be added later
print(f'number of countries unavailable: {y[~y["country"].isin(maternity['Country'])].dropna()['country'].count()}')
print(y[~y["country"].isin(maternity['Country'])].dropna()['country'])

number of countries unavailable: 6
7     Slovakia
10    Bulgaria
13      Cyprus
20     Croatia
22       Malta
23     Romania
Name: country, dtype: object


In [15]:
## adding the maternity leave data to the dataframe
df = pd.merge(df, maternity, how="left", left_on="country", right_on="Country").drop(columns="Country", axis=1)

## 

## Long term unemployment

In [16]:
ltunem = pd.read_csv('long_term_unemployment_males.csv')
ltunemf = pd.read_csv('long_term_unemployment_females.csv')

In [24]:
col = ['OBS_VALUE', 'TIME_PERIOD', 'Geopolitical entity (reporting)']
ltunem = ltunem[col].rename(columns={"OBS_VALUE": "male_unemployment_rate", "Geopolitical entity (reporting)": "country"}).drop("TIME_PERIOD", axis=1)
ltunemf = ltunemf[col].rename(columns={"OBS_VALUE": "female_unemployment_rate", "Geopolitical entity (reporting)": "country"}).drop("TIME_PERIOD", axis=1)

In [27]:
df = pd.merge(df, ltunemf, how="left", left_on="country", right_on="country")
df = pd.merge(df, ltunem, how="left", left_on="country", right_on="country")

## Gender pay gap in part-time employment

In [36]:
gpgptem = pd.read_csv('gender_pay_gap_part_time_employment.csv')

In [38]:
gpgptem = gpgptem[col].rename(columns={"OBS_VALUE": "gender_pay_gap_part_time_employment", "Geopolitical entity (reporting)": "country"}).drop("TIME_PERIOD", axis=1)

In [40]:
df = pd.merge(df, gpgptem, how="left", left_on="country", right_on="country")

## People having a long-standing illness or health problem by sex

In [55]:
health_problems_male = pd.read_csv('long_standing_illness_or_health_problem_males.csv')
health_problems_female = pd.read_csv('long_standing_illness_or_health_problem_females.csv')

In [58]:
col = ['OBS_VALUE', 'Geopolitical entity (reporting)']
health_problems_female = health_problems_female[col].rename(columns={"OBS_VALUE": "long_term_health_problems_female", "Geopolitical entity (reporting)": "country"})
health_problems_male = health_problems_male[col].rename(columns={"OBS_VALUE": "long_term_health_problems_male", "Geopolitical entity (reporting)": "country"})

In [61]:
df = pd.merge(df, health_problems_male, how="left", left_on="country", right_on="country")
df = pd.merge(df, health_problems_female, how="left", left_on="country", right_on="country")